# FUNCIONES API

In [ ]:
import pandas as pd
import json
import numpy as np

Cargar dataframes desde el directorio datasets

In [ ]:
reviews  = pd.read_parquet('../dataset/australian_user_reviews.parquet')
items =  pd.read_parquet('../dataset/australian_users_items.parquet')
games =  pd.read_parquet('../dataset/output_steam_games.parquet')
ranking_genre = pd.read_parquet('../dataset/ranking_genre.parquet')
usersXgenre = pd.read_parquet('../dataset/user_genre.parquet')

- def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [ ]:
def userdata(User_id:str):
    reviews  = pd.read_parquet('../dataset/australian_user_reviews.parquet')
    games =  pd.read_parquet('../dataset/output_steam_games.parquet')
    Consumo = 0
    user_data = reviews[reviews['user_id'] == User_id]
    item_count = user_data['user_id'].count()
    for i in user_data['item_id']:
        filtro = games[games['id'] == i]
        precio = filtro.iloc[0]['price'] - filtro.iloc[0]['discount_price']
        Consumo = Consumo + precio
    recomemdaciones = reviews[reviews['user_id'] == User_id]
    cierto = recomemdaciones[recomemdaciones['recommend'] == True].count()
    porcentaje = cierto[0]  / item_count
    resultado = {"Consumo del usuario": Consumo, "% de recomendación": porcentaje * 100, "Cantidad de items": item_count}
    del reviews
    del games
    return resultado

In [ ]:
userdata('kimjongadam')

- def countreviews( YYYY-MM-DD y YYYY-MM-DD : str ): Cantidad de usuarios que realizaron reviews entre las fechas dadas y, el porcentaje de recomendación de los mismos en base a reviews.recommend.

In [ ]:
def countreviews(fecha1:str,fecha2:str):
    reviews  = pd.read_parquet('../dataset/australian_user_reviews.parquet')
    if fecha1 > fecha2:
        return {"Error":"Ingrese correctamente las fechas"}
    else:
        filtro = reviews[(reviews['posted_date'] >= fecha1) & (reviews['posted_date'] <= fecha2) ]
        cierto = filtro[filtro['recommend'] == True].count()
        Tot_rev = filtro['recommend'].count()
        porcentaje = cierto['recommend'] / Tot_rev
    resultado = {"Cantidad de Reviews": int(filtro['recommend'].count()), "% de recomendaciones": round(float(porcentaje),2) * 100}
    del reviews
    return resultado

In [ ]:
countreviews('2010-10-15','2010-12-20')

- def genre( género : str ): Devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.

Generar ranking_genre.parquet

In [ ]:
# No ejecutar esta celda en render ya que se consume alrededor de 2 Gb de memoria y la versión gratis tiene solo 512 Mb
# Separar lista de generos
games_exploded = games.explode('genres')
# Combinar DF's games_explode con items coincidiendo por id    
merged_data = games_exploded.merge(items, left_on='id', right_on='item_id', how='inner')
# Sumar playtime_forever por género, este dataframe supera el gigabyte de memoria, por ello es necesario almacenar el df ranking_genre
genre_playtime = merged_data.groupby('genres')['playtime_forever'].sum().reset_index()
# Ordenar de mayor a menor sobre 'playtime_forever'
ranking_genre = genre_playtime.sort_values(by='playtime_forever', ascending=False)
ranking_genre = ranking_genre.reset_index(drop=True)
# Guardar df en formato parquet
ranking_genre.to_parquet('../dataset/ranking_genre.parquet', engine='pyarrow', compression='snappy')

In [ ]:
def genre(genero:str):
    ranking_genre = pd.read_parquet('../dataset/ranking_genre.parquet')
    resultado = {"Puesto numero:": ranking_genre[ranking_genre['genres'] == genero].index[0] + 1 }
    del ranking_genre
    return resultado

In [ ]:
genre('Photo Editing')

- def userforgenre( género : str ): Top 5 de usuarios con más horas de juego en el género dado, con su URL (del user) y user_id.

In [ ]:
# Crear df user_genre.parquet

# Creando df con las columnas 'genres', 'playtime_forever' y 'user_id_y'
usersXgenre = merged_data[['genres','playtime_forever','user_id_y','user_url']]
# Agrupando por usuario y genero
usersXgenre = usersXgenre.groupby(['genres','user_id_y','user_url'])['playtime_forever'].sum().reset_index()
# Guardar df en formato parquet
usersXgenre.to_parquet('../dataset/user_genre.parquet', engine='pyarrow', compression='snappy')

In [ ]:
def userforgenre(genero:str):
    usersXgenre = pd.read_parquet('../dataset/user_genre.parquet')
    filtro = usersXgenre[usersXgenre['genres'] == genero].sort_values(by='playtime_forever', ascending=False).head(5)
    lista = []
    j = 0
    for i in filtro['user_id_y']:
        filtro2 = filtro[filtro['user_id_y'] == i]
        res = {"Posicion": j + 1, "Usuario_Id":i,"Url_Usuario":filtro2['user_url'].iloc[0]}
        lista.append(res)
        j = j + 1
    del usersXgenre
    return lista

In [ ]:
userforgenre('RPG')


- def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de salida:

|Activision | |
|:-------:|:----------:|	
| Año | Contenido Free |
| 2023 |	27% |
| 2022 |	25% |
| xxxx |	xx% |

In [ ]:
def developer(desarrollador:str):
    games =  pd.read_parquet('../dataset/output_steam_games.parquet')
    filtro = games[games['developer'] == desarrollador].dropna(subset=['release_date'])
    filtro = filtro[filtro['release_date'].str.match(r'\d{4}-\d{2}-\d{2}', na=False)]
    filtro['release_date'] = pd.to_datetime(filtro['release_date'], format="%Y-%m-%d")
    filtro_free = filtro[filtro['price'] == 0]
    total = filtro.groupby(filtro['release_date'].dt.year)['price'].count()
    free = filtro_free.groupby(filtro_free['release_date'].dt.year)['price'].count()
    resultado = pd.DataFrame({'free': total, 'total': free})
    resultado = resultado.fillna(0)
    resultado['free'] = resultado['free'].astype(int)
    resultado['total'] = resultado['total'].astype(int)
    lista = []
    
    for i in range(len(resultado)):
        a = resultado['free'].iloc[i]
        b = resultado['total'].iloc[i]
        if b == 0:
            porcentaje = None
        else:
            try:
                porcentaje = round(b * 100 / a, 2)
            except ZeroDivisionError:
                porcentaje = 0.0  # O cualquier otro valor predeterminado
            
        res = {"Año": int(resultado.index[i]), "Contenido free": int(a), "% free": porcentaje}
        lista.append(res)
    
    # Limpieza de variables intermedias
    del games
    del filtro
    del filtro_free
    
    return lista

In [ ]:
developer('SmiteWorks USA, LLC')

- def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

--Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [ ]:
# Generar parquet sentiments
merged_sentiments = games.merge(reviews, left_on='id', right_on='item_id', how='inner')
sentiments = merged_sentiments[['release_date','sentiment_analysis']]
sentiments = sentiments[sentiments['release_date'].str.match(r'\d{4}-\d{2}-\d{2}', na=False)]
sentiments['release_date'] = pd.to_datetime(sentiments['release_date'], format="%Y-%m-%d")
sentiments.to_parquet('../dataset/sentiments.parquet', engine='pyarrow', compression='snappy')

In [ ]:
def sentiment_analysis(anio:int):
    sentiments  = pd.read_parquet('..\dataset\sentiments.parquet')
    filtro = sentiments[sentiments['release_date'].dt.year == anio]
    filtro['sentiment_analysis'] = filtro['sentiment_analysis'].astype(int)
    filtro['release_date'] = filtro['release_date'].astype('int64').astype('int32')
    resumen = filtro.groupby(filtro['sentiment_analysis'])['release_date'].count().to_dict()
    resultado = {"Negative=": resumen.get(0, 0), "Neutral=": resumen.get(1, 0), "Positive=": resumen.get(2, 0)}
    del sentiments
    return resultado

In [ ]:
sentiment_analysis(2011)

# Modelo de aprendizaje automático

In [ ]:
import pandas as pd
from fastapi import FastAPI
import json
import csv
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
reviews  = pd.read_parquet('../dataset/australian_user_reviews.parquet')
items =  pd.read_parquet('../dataset/australian_users_items.parquet')
games =  pd.read_parquet('../dataset/output_steam_games.parquet')
ranking_genre = pd.read_parquet('../dataset/ranking_genre.parquet')
usersXgenre = pd.read_parquet('../dataset/user_genre.parquet') 
sentiments = pd.read_parquet('../dataset/sentiments.parquet')

- def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

In [ ]:
def list_to_string(tag_list):
    if tag_list is None:
        return ''
    elif isinstance(tag_list, list):
        return ' '.join(tag_list)
    else:
        # En caso de que no sea una lista, puedes manejarlo de acuerdo a tus necesidades.
        return str(tag_list)


def recomendaciopm_juego(game_id:int):
    games =  pd.read_parquet('../dataset/output_steam_games.parquet')
    # Filtra el DataFrame para obtener el juego objetivo por su ID
    target_game = games[games['id'] == game_id]

    # Extrae las etiquetas (tags) del juego objetivo
    target_tags = target_game['tags'].values[0]

    # Convierte las etiquetas en una cadena de texto
    target_tags_str = ' '.join(target_tags)

    # Inicializa el vectorizador de recuento para las etiquetas
    vectorizer = CountVectorizer()

    # Aplica el vectorizador de recuento a todas las etiquetas en el DataFrame
    games['tags_str'] = games['tags'].apply(list_to_string)
    tag_matrix = vectorizer.fit_transform(games['tags_str'])

    # Calcula la similitud del coseno entre las etiquetas del juego objetivo y todos los juegos en el DataFrame
    cosine_sim = cosine_similarity(tag_matrix)

    # Obtiene las puntuaciones de similitud del juego objetivo con todos los juegos
    target_sim_scores = cosine_sim[games.index[games['id'] == game_id]]

    # Ordena los juegos por similitud y obtiene los juegos similares (excluyendo el juego objetivo)
    similar_games_indices = target_sim_scores.argsort()[0][-5 - 1:-1][::-1]

    # Crea una lista de juegos similares
    similar_games = games.iloc[similar_games_indices]

    return similar_games


In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

def list_to_string(tag_list):
    if tag_list is None:
        return ''
    elif isinstance(tag_list, list):
        return ' '.join(tag_list)
    else:
        # En caso de que no sea una lista, puedes manejarlo de acuerdo a tus necesidades.
        return str(tag_list)

games =  pd.read_parquet('../dataset/output_steam_games.parquet')
filtro = games[games['id'] == '610660']
tags = filtro['tags'].values[0]
tags_str = ' '.join(tags)
vectorizer = CountVectorizer()
games['tags_str'] = games['tags'].apply(list_to_string)
tag_matrix = vectorizer.fit_transform(games['tags_str'])
cosine_sim = cosine_similarity(tag_matrix)
target_sim_scores = cosine_sim[games.index[games['id'] == game_id]]
similar_games_indices = target_sim_scores.argsort()[0][-5 - 1:-1][::-1]
similar_games = games.iloc[similar_games_indices]

In [28]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
result_data = []

def list_to_string(tag_list):
    if tag_list is None:
        return ''
    elif isinstance(tag_list, list):
        return ' '.join(tag_list)
    else:
        return str(tag_list)

# Cargar el DataFrame completo
games = pd.read_parquet('../dataset/output_steam_games.parquet')

# Seleccionar solo las columnas relevantes
games = games[['id', 'tags']]

# Dividir el DataFrame en 10 partes iguales
num_parts = 20
part_size = len(games) // num_parts

# Crear un DataFrame vacío para almacenar los resultados finales
result_df = pd.DataFrame(columns=['game_id', 'similar_game_id', 'similarity'])

# Crear un diccionario para mapear los índices originales a los nuevos índices
index_mapping = {game_index: i for i, game_index in enumerate(games.index)}

# Iterar sobre las partes del DataFrame
for i in range(num_parts):
    start_idx = i * part_size
    end_idx = (i + 1) * part_size if i < num_parts - 1 else len(games)
    
    # Obtener la parte actual del DataFrame
    part_games = games.iloc[start_idx:end_idx].copy()  # Hacer una copia
    
    # Crear una matriz de etiquetas para la parte actual
    vectorizer = CountVectorizer()
    part_games['tags_str'] = part_games['tags'].apply(list_to_string)
    tag_matrix = vectorizer.fit_transform(part_games['tags_str'])
    
    # Calcular la similitud coseno solo para pares con similitud >= 0.8
    cosine_sim = cosine_similarity(tag_matrix)
    threshold = 0.8
    cosine_sim[cosine_sim < threshold] = 0
    
    # Encontrar las 5 mejores coincidencias para cada juego en la parte actual
    for game_index, row in part_games.iterrows():
        print("game_index:", game_index)
        game_id = row['id']
        target_sim_scores = cosine_sim[index_mapping[game_index]]
        similar_games_indices = target_sim_scores.argsort()[-5 - 1:-1][::-1]
        
        # Almacenar los resultados en el DataFrame de resultados
        for similar_game_index in similar_games_indices:
            similar_game_id = games.iloc[similar_game_index]['id']
            result_data.append({
                'game_id': game_id,
                'similar_game_id': similar_game_id,
                'similarity': target_sim_scores[similar_game_index]
            })

# Convertir la lista de diccionarios en un DataFrame
result_df = pd.DataFrame(result_data)

# Mostrar las 5 mejores coincidencias para cada juego
print(result_df.head())


game_index: 88310
game_index: 88311
game_index: 88312
game_index: 88313
game_index: 88314
game_index: 88315
game_index: 88316
game_index: 88317
game_index: 88318
game_index: 88319
game_index: 88320
game_index: 88321
game_index: 88322
game_index: 88323
game_index: 88324
game_index: 88325
game_index: 88326
game_index: 88327
game_index: 88328
game_index: 88329
game_index: 88330
game_index: 88331
game_index: 88332
game_index: 88333
game_index: 88334
game_index: 88335
game_index: 88336
game_index: 88337
game_index: 88338
game_index: 88339
game_index: 88340
game_index: 88341
game_index: 88342
game_index: 88343
game_index: 88344
game_index: 88345
game_index: 88346
game_index: 88347
game_index: 88348
game_index: 88349
game_index: 88350
game_index: 88351
game_index: 88352
game_index: 88353
game_index: 88354
game_index: 88355
game_index: 88356
game_index: 88357
game_index: 88358
game_index: 88359
game_index: 88360
game_index: 88361
game_index: 88362
game_index: 88363
game_index: 88364
game_index

IndexError: index 1606 is out of bounds for axis 0 with size 1606

: 

In [27]:
print("Tamaño de cosine_sim:", cosine_sim.shape)

Tamaño de cosine_sim: (1606, 1606)


In [ ]:
games['id'].info()